In [2]:
import numpy as np
import matplotlib

import matplotlib.pyplot as plt

import matplotlib.pyplot as figure
import seaborn as sns  # you might not have this installed -- it makes graphs prettier and sometimes easier
# %matplotlib inline 
import umap
import tldextract
from sklearn.preprocessing import StandardScaler
import pandas as pd
import publicsuffixlist as psl
import os
from tqdm import tqdm_notebook
from sklearn import preprocessing

In [3]:
another_data_part = pd.read_parquet("data_20180705.parquet", engine= "fastparquet")

In [4]:
another_data_part['pd_ts'] = pd.to_datetime(another_data_part.timestamp, unit='s')  # leave off the unit='s' and your dates will be wrong
# another_data_part = another_data_part.set_index('pd_ts').sort_index()
another_data_part['date'] = another_data_part['pd_ts'].dt.date

In [5]:
another_data_part['date'] = another_data_part['date'].astype(str)

In [6]:
another_data_part.date.value_counts()

2018-07-05    5248451
2018-07-06      43405
2018-07-04         11
Name: date, dtype: int64

In [7]:
another_data_part.domain.value_counts()


airbnb.com             1577304
mheginc.com            1454389
tinder.com              991028
americanexpress.com     791435
withairbnb.com          477711
Name: domain, dtype: int64

In [25]:
telstra_aug1 = another_data_part[(another_data_part.domain == 'tinder.com') & (another_data_part.date =='2018-07-06')]

In [26]:
telstra_aug1

,timestamp,qname,qip,rip,rcode,qtype,domain,ancount,pd_ts,date
76815,1530835209,go.tinder.com.,68.105.28.141,205.251.193.47,0,28,tinder.com,1,2018-07-06 00:00:09,2018-07-06
77048,1530835272,go.tinder.com.,77.88.56.70,205.251.197.203,0,1,tinder.com,1,2018-07-06 00:01:12,2018-07-06
77062,1530835261,tinder.com.,68.105.29.43,205.251.194.172,0,1,tinder.com,4,2018-07-06 00:01:01,2018-07-06
77205,1530835362,tinder.com.,68.105.28.173,205.251.194.172,0,28,tinder.com,0,2018-07-06 00:02:42,2018-07-06
77290,1530835348,go.tinder.com.,68.105.29.206,205.251.194.172,0,28,tinder.com,1,2018-07-06 00:02:28,2018-07-06
77428,1530835391,tinder.com.,68.105.29.235,205.251.197.203,0,28,tinder.com,0,2018-07-06 00:03:11,2018-07-06
77496,1530835425,tinder.com.,68.105.28.204,205.251.198.35,0,1,tinder.com,4,2018-07-06 00:03:45,2018-07-06
77646,1530835466,www.tinder.com.,68.105.29.44,205.251.194.172,0,1,tinder.com,4,2018-07-06 00:04:26,2018-07-06
77689,1530835478,tinder.com.,77.88.56.78,205.251.198.35,0,1,tinder.com,4,2018-07-06 00:04:38,2018-07-06
77690,1530835453,tinder.com.,68.105.28.76,205.251.198.35,0,1,tinder.com,4,2018-07-06 00:04:13,2018-07-06


In [27]:
subdomains = []

# for dom in dataAttackRemotlyAnwhere['qname']:
for dom in telstra_aug1['qname']:
    labels = tldextract.extract(dom)
    if labels.subdomain != "":
        subdomains.append(labels.subdomain)

In [28]:
subdomains = list(set(subdomains))

In [29]:
telstra_aug1 = pd.DataFrame(columns = ['prefix','pref_len', 'bigram_pref','label', 'domain'])

In [30]:
telstra_aug1['prefix'] = subdomains

In [31]:
telstra_aug1['bigramPref'] = ["^{0}$".format(element) for element in subdomains] 

In [32]:
telstra_aug1['pref_len'] = [len(prefix) for prefix in subdomains]

In [33]:
telstra_aug1['label'] = 1

In [34]:
telstra_aug1['domain'] = 'tinder.com_Jul06'

In [35]:
telstra_aug1.to_csv("differentDaysAttacks/tinder_july06.csv", index = False)